노트북(.ipynb)을 ch10 폴더 최상단에 두고 실행하세요

## 서버 시작 / 종료 함수

In [7]:
!pip install mcp fastmcp
!pip install llama-index-tools-mcp

  Using cached llama_index_core-0.14.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached llama_index_workflows-2.2.0-py3-none-any.whl.metadata (6.4 kB)
Using cached llama_index_core-0.14.1-py3-none-any.whl (11.9 MB)
Using cached llama_index_workflows-2.2.0-py3-none-any.whl (56 kB)
  Attempting uninstall: llama-index-workflows
    Found existing installation: llama-index-workflows 1.3.0
    Uninstalling llama-index-workflows-1.3.0:
      Successfully uninstalled llama-index-workflows-1.3.0
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.52.post1
    Uninstalling llama-index-core-0.12.52.post1:
      Successfully uninstalled llama-index-core-0.12.52.post1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [llama-index-tools-mcp]a-index-core]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-hugging

In [13]:
# 헬퍼: 백그라운드 실행/종료 + PID/로그 관리
import os, sys, json, signal, subprocess
from pathlib import Path

ROOT = Path.cwd()
PID_DIR = ROOT / ".pids"
LOG_DIR = ROOT / ".logs"
PID_DIR.mkdir(exist_ok=True)
LOG_DIR.mkdir(exist_ok=True)

def _env():
    env = os.environ.copy()
    # .env 사용 시 자동로드(없으면 무시)
    try:
        from dotenv import load_dotenv
        load_dotenv(ROOT / ".env")
        env.update(os.environ)
    except Exception:
        pass
    return env

def start_bg(script_rel: str, name: str):
    """script_rel: 예) 'server.py' / 'document_search/server.py' 등
       name: 식별용 태그(로그/피드파일명) 예) 'basic', 'docs', 'weather'
    """
    script_abs = (ROOT / script_rel).resolve()
    if not script_abs.exists():
        raise FileNotFoundError(script_abs)
    pid_file = PID_DIR / f"{name}.pid"
    log_file = LOG_DIR / f"{name}.log"
    fout = open(log_file, "a", buffering=1)             # 로그 파일에 바로 기록
    p = subprocess.Popen(
        [sys.executable, str(script_abs)],
        cwd=str(script_abs.parent),
        env=_env(),
        stdout=fout,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    pid_file.write_text(str(p.pid))
    print(f"[START] {name}: pid={p.pid}, log={log_file}")

def stop_bg(name: str):
    """name으로 기록된 프로세스 종료"""
    pid_file = PID_DIR / f"{name}.pid"
    if not pid_file.exists():
        print(f"[INFO] no pid for {name}")
        return
    pid = int(pid_file.read_text().strip() or "0")
    try:
        os.kill(pid, signal.SIGTERM)
        print(f"[STOP] {name}: SIGTERM sent to pid={pid}")
    except ProcessLookupError:
        print(f"[WARN] {name}: pid {pid} not found (already dead?)")
    except Exception as e:
        print(f"[WARN] {name}: kill failed -> {e}")
    try:
        pid_file.unlink()
    except Exception:
        pass

def tail_log(name: str, n: int = 80):
    """최근 로그 확인"""
    log_file = LOG_DIR / f"{name}.log"
    if not log_file.exists():
        print(f"[INFO] no log for {name}")
        return
    # 간단 tail
    with open(log_file, "r") as f:
        lines = f.readlines()[-n:]
    print("".join(lines))

## ch10 server.py 실행 / 종료

In [14]:
# 기본 MCP 서버 백그라운드 실행
start_bg("server.py", name="basic")

# 필요하면 최근 로그만 확인
tail_log("basic", n=50)

[START] basic: pid=99765, log=/Users/paesir/Desktop/git/llama-index/ch10/.logs/basic.log



In [18]:
# List Tools
env = _load_env()
p = subprocess.run([sys.executable, str((ROOT / "list_tools.py").resolve())],
                   cwd=str(ROOT),
                   env=env, capture_output=True, text=True)
print(p.stdout or p.stderr)

{
  "name": "search_llama_docs",
  "description": "llamaindex 문서에서 질의에 맞는 내용을 검색합니다.",
  "parameters": {
    "properties": {
      "query": {
        "description": "",
        "title": "Query",
        "type": "string"
      }
    },
    "required": [
      "query"
    ],
    "type": "object"
  }
}
{
  "name": "search_github_docs",
  "description": "깃허브 문서에서 질의에 맞는 내용을 검색합니다.",
  "parameters": {
    "properties": {
      "query": {
        "description": "",
        "title": "Query",
        "type": "string"
      }
    },
    "required": [
      "query"
    ],
    "type": "object"
  }
}



In [16]:
# 서버 종료
stop_bg("basic")

[STOP] basic: SIGTERM sent to pid=99765


## document 서버 실행 / 종료

In [23]:
# ▶ Start Document Search Server
# MCP 서버 백그라운드 실행
start_bg("document_search/server.py", name="document")

# 필요하면 최근 로그만 확인
tail_log("document", n=50)

[START] document: pid=99924, log=/Users/paesir/Desktop/git/llama-index/ch10/.logs/document.log
INFO:     Started server process [99872]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 48] error while attempting to bind on address ('127.0.0.1', 8000): [errno 48] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.



In [20]:
# (Optional) List Tools for Document Search
env = _load_env()
list_path = ROOT / "document_search" / "list_tools.py"
if list_path.exists():
    p = subprocess.run([sys.executable, str(list_path.resolve())],
                       cwd=str(list_path.parent),
                       env=env, capture_output=True, text=True)
    print(p.stdout or p.stderr)
else:
    print("[INFO] document_search/list_tools.py not found.")

{
  "name": "search_llama_docs",
  "description": "llamaindex 문서에서 질의에 맞는 내용을 검색합니다.",
  "parameters": {
    "properties": {
      "query": {
        "description": "",
        "title": "Query",
        "type": "string"
      }
    },
    "required": [
      "query"
    ],
    "type": "object"
  }
}
{
  "name": "search_github_docs",
  "description": "깃허브 문서에서 질의에 맞는 내용을 검색합니다.",
  "parameters": {
    "properties": {
      "query": {
        "description": "",
        "title": "Query",
        "type": "string"
      }
    },
    "required": [
      "query"
    ],
    "type": "object"
  }
}



In [21]:
# ▶ Run Document Search Client
env = _load_env()
client_path = ROOT / "document_search" / "client.py"
if client_path.exists():
    p = subprocess.run([sys.executable, str(client_path.resolve())],
                       cwd=str(client_path.parent),
                       env=env, capture_output=True, text=True)
    print(p.stdout or p.stderr)
else:
    print("[INFO] document_search/client.py not found.")


=== 에이전트 응답 ===

Llama-Index를 활용하여 문서를 검색하는 방법은 다음과 같습니다:

1. **파일 경로 방문**: LlamaIndex 문서가 저장된 파일 경로를 방문하여 관련 정보를 찾습니다.
2. **내용 탐색**: 문서의 내용을 살펴보며 LlamaIndex와 관련된 정보를 검색합니다.
3. **생태계 탐색**: LlamaIndex 생태계 및 관련 프로젝트를 탐색하여 추가적인 리소스를 찾습니다.

이러한 방법을 통해 Llama-Index와 관련된 유용한 정보를 효과적으로 검색할 수 있습니다.



In [24]:
# 서버 종료
stop_bg("document")

[STOP] document: SIGTERM sent to pid=99924


## weather 서버 실행 / 종료

In [25]:
# ▶ Start Weather Server
# MCP 서버 백그라운드 실행
start_bg("weather/weather-server.py", name="weather")

# 필요하면 최근 로그만 확인
tail_log("weather", n=50)

[START] weather: pid=99964, log=/Users/paesir/Desktop/git/llama-index/ch10/.logs/weather.log



In [26]:
# ▶ Run Weather Client
env = _load_env()
client_path = ROOT / "weather" / "weather-client.py"
p = subprocess.run([sys.executable, str(client_path.resolve())],
                   cwd=str(client_path.parent),
                   env=env, capture_output=True, text=True)
print(p.stdout or p.stderr)

사용자: 오늘 날씨 어때?
에이전트: 어떤 도시의 날씨를 알고 싶으신가요? 도시명을 말씀해 주시면 그에 맞는 정보를 제공해 드리겠습니다.
사용자: 서울이에요
에이전트: 서울의 날씨 정보를 알고 싶으신가요? 오늘 또는 내일 중 어떤 날씨 정보를 원하시는지 말씀해 주세요.
사용자: 내일 부산 날씨는?
에이전트: 죄송하지만, 내일 부산의 날씨에 대한 정보를 제공할 수 없습니다. 다른 질문이 있으시면 도와드리겠습니다!



In [27]:
# 서버 종료
stop_bg("weather")

[STOP] weather: SIGTERM sent to pid=99964


## decorater 실행

In [28]:
# ▶ Run decorator sample
path = ROOT / "decorater_sample.py"
if path.exists():
    env = _load_env()
    p = subprocess.run([sys.executable, str(path.resolve())],
                       cwd=str(path.parent),
                       env=env, capture_output=True, text=True)
    print(p.stdout or p.stderr)
else:
    print("[INFO] decorater_sample.py not found.")

Calling function: greet
Hello, Alice!



## 모든 서버 종료

In [29]:
stop_all()

[DONE] stop_all
